In [1]:
import re
import os
import groq as Groq
from dotenv import load_dotenv

## Iniciando o Agente e fazendo as conexões necessárias

In [2]:
#para rodar tudo, é necessário ter uma chave API do Groq, que pode ser obtida em https://groq.io/developers
#a minha está em um arquivo dotenv para que não seja exposta, mas você pode colocar a sua diretamente no código
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
client = Groq.Client(
    api_key=groq_api_key,
)

#note que o groq pega uma lista de mensagens, não uma única string
chat_completion = client.chat.completions.create(
    #primeiro parametro é a lista de mensagens
    messages=[

        {

            "role": "user",

            "content": "Explain the importance of fast language models",

        }

    ],
    #segundo parametro é o modelo de linguagem que será usado (é possível setar outro modelo que a API hospeda)
    model="llama-3.3-70b-versatile",
    
)


print(chat_completion.choices[0].message.content)


Fast language models are crucial in the field of natural language processing (NLP) due to their ability to quickly and accurately process and generate human-like language. The importance of fast language models can be seen in several areas:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation software. These models can quickly respond to user queries, making the interaction more natural and efficient.
2. **Improved User Experience**: Fast language models can process and respond to user input rapidly, reducing latency and improving the overall user experience. This is particularly important in applications where timely responses are critical, such as customer service chatbots.
3. **Scalability**: Fast language models can handle a large volume of requests and process them quickly, making them scalable for large-scale applications. This is essential for applications that need to handle multiple co

In [4]:
class Agente:
    def __init__(self, client, sistema):
        self.client = client
        self.sistema = sistema
        self.mensagem = []
        if self.mensagem is not None:
            self.mensagem.append({
                "role": "system",
                "content": self.sistema
            })

    def __call__(self, mensagem=""):
        if mensagem:
            self.mensagem.append({
                "role": "user",
                "content": mensagem
            })
        resultado = self.executar()
        self.mensagem.append({
            "role": "assistant",
            "content": resultado
        })
        return resultado

    def executar(self):
        completion = client.chat.completions.create(
            messages=self.mensagem,
            model='llama-3.3-70b-versatile'
        )
        return completion.choices[0].message.content


Até aqui eu modifiquei pouquissimas coisas do código do vídeo pois é apenas a criação da classe de agente e inicialização do groq

## Montando o prompt
Nesse passo, eu montei o prompt seguindo a maneira que o vídeo de base fez, porém preferi fazer em português pois fica algo mais orgânico e fácil de entender. Mesmo não tendo problemas para fazer em inglês, optei por fazer em PT. Basicamente eu estou pedindo para o modelo fazer uma transformação básica de unidades de temperaturas com alguns cálculos matemáticos envolvidos. Tive algumas tentativas frustradas antes dessa ideia pois estava tentando fazer algo muito complexo e extenso, e percebi que seria inviável concretizar isso utilizando somente python puro e um pouco de ReGEx. Mesmo assim, o prompt ficou muito coeso e acredito ter sido a melhor escolha de palavras.

In [5]:
system_prompt = """
Você vai operar em um loop de Pensamento,Ação, PAUSE, Observação.
Ao final do loop, você emitirá uma Resposta.
Use "Pensamento" para descrever seus pensamentos sobre a pergunta que foi feita, retorne PAUSE.
Use "Ação" para executar uma das ações disponíveis e, em seguida, retorne PAUSE.
"Observação" será o resultado da execução dessa ação.

Suas ações disponíveis são:

get_scale:
e.g. get_scale: "Celsius" 
Obtém a escala de temperatura.

scale_wanted:
e.g. scale_wanted: "Fahrenheit"
Obtém a escala de temperatura que irá ser transformada.

transform:
e.g. transform: "get_scale para scale_wanted"
Transforma uma unidade de temperatura para outra.

calculate:
e.g. calculate: 100 * 1.8 + 32 
Calcula o numero que foi pedido com base na formula de transformação de cada unidade de temperatura. Utilize calculadora em python e tenha certeza de utilizar a sintaxe em float corretamente, se necessário.

Exemplo de sessão:
Pergunta: Qual é a temperatura de 100 graus Celsius em Fahrenheit, multiplicado por 2?
Pensamento: Preciso transformar 100 graus Celsius em Fahrenheit.
Ação: transform: "Celsius para Fahrenheit"
PAUSE

Você será chamado novamente

(Observação recebida: 100 graus Celsius é igual a 212 graus Fahrenheit.)

Pensamento: Agora preciso multiplicar 212 por 2.
Ação: calculate: 212 * 2
PAUSE

Se você tiver a resposta, retorne-a como Resposta.

Resposta: 424° Fahrenheit

Agora é sua vez!
""".strip()


## Funções do agente
Aqui estão as funções lógicas que o agente irá fazer para conseguir realizar o pedido de maneira correta. Aqui temos 4 funções, todas fazendo exatamente o que a teoria do prompt pede.

Primeiro, temos as funções de coleta de escala: get_scale e scale_wanted. As funções são utilizadas para entender qual a escala que a pessoa pegou e transformar para a escala solicitada. Note que eu acabei tendo que fazer um replace nas duas funções, esse foi um erro que me custou praticamente o final de semana inteiro, pois o agente não estava reconhecendo as unidades como string normal (Celsius, Fahrenheit, Kelvin), mas sim como string e aspas ("Celsius", "Fahrenheit", "Kelvin"), e isso fazia com que a função de transformação não pudesse realizar sua ação, travando a operação no loop que será comentado mais a frente

Então temos a função transform, que é a função que fará a transformação desejada da escala desejada para a escala alvo. Essa transformação faz um fetch da current_scale e da target_scale, e é possível pois, mesmo se for inserido com letra maiuscula, a função nativa .lower é capaz de padronizar tudo e seguir com o planejado.

Por último temos a função calculate que é um operador lógico para fazer os demais cálculos desejados.

In [12]:
def get_scale(unidade_str):
    global current_scale
    unidade_str = unidade_str.replace('"', '').strip()
    current_scale = unidade_str.strip().lower()
    return f"Unidade de origem definida como {current_scale.capitalize()}."

def scale_wanted(unidade_str):
    global target_scale
    unidade_str = unidade_str.replace('"', '').strip()
    target_scale = unidade_str.strip().lower()
    return f"Unidade de destino definida como {target_scale.capitalize()}."

def transform(value):
    if current_scale is None or target_scale is None:
        raise ValueError("As escalas de origem e destino precisam ser definidas antes da transformação.")
    
    transformacao_str = f"{current_scale} para {target_scale}"
    
    if transformacao_str == "celsius para fahrenheit":
        return value * 1.8 + 32
    elif transformacao_str == "fahrenheit para celsius":
        return (value - 32) / 1.8
    elif transformacao_str == "celsius para kelvin":
        return value + 273.15
    elif transformacao_str == "kelvin para celsius":
        return value - 273.15
    elif transformacao_str == "fahrenheit para kelvin":
        return (value - 32) / 1.8 + 273.15
    elif transformacao_str == "kelvin para fahrenheit":
        return (value - 273.15) * 1.8 + 32
    else:
        raise ValueError("Transformação de temperatura inválida.")

def calculate(formula):
    return eval(formula)

## Loop
Aqui possuímos o coração do nosso agente, pois é onde faremos o loop de ações em que ele percorrerá até encontrar a resposta desejada. Aqui o agente percorre as etapas de Pensamento, Ação, Observação e Resposta. Como foi solicitado no prompt, o agente não deve fazer todas as ações de uma vez, portanto ele percorre um pensamento, uma ação, pausa e observa para ver se está tudo nos conformes. Então ele segue para outro pensamento e ação, e continua nesse loop até que seja concluída a solicitação feita. Nesse caso, é importante considerar o fato de que devemos deixar explícito em código um número máximo de iterações que o loop deve fazer para que ele não fique rodando infinitamente, caso não seja possível realizar o pedido.

In [ ]:
def agente_loop(max_iter, system_prompt, pergunta):
    agente = Agente(client, system_prompt)
    ferramentas = {
        "get_scale": get_scale,
        "scale_wanted": scale_wanted,
        "transform": transform,
        "calculate": calculate,
    }
    prox_prompt = pergunta
    i = 0
    
    while i < max_iter:
        i += 1
        resultado = agente(prox_prompt)
        print(resultado)

        if "PAUSE" in resultado and "Ação" in resultado:
            acao = re.findall(r"Ação: ([a-z_]+): (.+)", resultado, re.IGNORECASE)
            if acao:
                ferramenta = acao[0][0]
                argumento = acao[0][1]
                if ferramenta in ferramentas:
                    if ferramenta == "transform":
                        resultado_exec = ferramentas[ferramenta](100)
                    else:
                        resultado_exec = ferramentas[ferramenta](argumento)
                    prox_prompt = f"Observação recebida: {resultado_exec}"
                else:
                    prox_prompt = "Por favor, insira uma ação válida."
                print(prox_prompt)
                continue

        if "Resposta" in resultado:
            break


In [15]:
agente_loop(10, system_prompt, "Qual é a temperatura de 2 graus Celsius em Kelvin, multiplicado por 15?")

Pensamento: Preciso transformar 2 graus Celsius em Kelvin, e em seguida multiplicar o resultado por 15. Para começar, vou verificar a escala de temperatura que estou começando. 

Ação: get_scale: "Celsius"
PAUSE
Observação recebida: Unidade de origem definida como Celsius.
Pensamento: Agora que sei que estou começando com a escala Celsius, preciso definir a escala de destino, que é Kelvin. 

Ação: scale_wanted: "Kelvin"
PAUSE
Observação recebida: Unidade de destino definida como Kelvin.
Pensamento: Com as escalas de origem e destino definidas, posso agora transformar 2 graus Celsius em Kelvin. 

Ação: transform: "Celsius para Kelvin"
PAUSE
Observação recebida: 373.15
Pensamento: Agora que sei que 2 graus Celsius equivalem a 275,15 Kelvin, mas aqui está 373,15, considerei o zero absoluto para a equação, sendo assim 273,15 + 2 = 275,15, agora estou errado, pois considero 0 absoluto em vez do valor, o valor correto é 2 + 273,15 = 275,15, porém recebi 373,15 que é o valor de 100°C + 273,15